# This project helps locate resturants and food places in Toronto. This project will first find the top 2 venues for each neighborhood in Toronto, then will locate a place to begin an investment in an arabian sweet bakary.

### Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
#! pip install html5lib
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

## Get the data from wikipedia

In [3]:
#Accessing the Data
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL)

## Parse through the data saving the values of postal code, borough and neighborhood

In [4]:
soup = BeautifulSoup(r.content, 'html5lib')
#Collecting the Data
PostalCode=[]
Borough=[]
Neighborhood=[]
rows=[]
cnum=1 # parsing by column number since its only 3 columns 
for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            if cnum==1:
                PostalCode.append(cell.string)
                cnum=2
            elif cnum==2 :
                if cell.string=='Not assigned\n':
                    Borough.append(None)
                    cnum=3
                else:
                    Borough.append(cell.string)
                    cnum=3
            else:
                Neighborhood.append(cell.string)
                cnum=1

## Merge the data collected

In [5]:
#Merging the Data
code=[PostalCode,Borough,Neighborhood]
codes=pd.DataFrame(code)
codes=codes.transpose()

# Formating data into dataframes with a coloumn 
#tor refers to toronto
tor=pd.DataFrame(data=codes)
tor.columns=['PostalCode','Borough','Neighborhood']
tor.head()

,PostalCode,Borough,Neighborhood
0,M1A,None,Not assigned
1,M2A,None,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Data Wrangling

In [6]:
#removing any none values
tor.dropna(subset=['Borough'],axis=0,inplace=True)
tor=tor.reset_index(drop=True)
tor.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Load Latitude and Longitude coordinates of Postal Code

In [7]:
latlong=pd.read_csv('http://cocl.us/Geospatial_data')
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Data Wrangling

In [8]:
latlong.columns=['PostalCode','Latitude','Longitude']
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
#removing unnecessary formatting
tor=tor.replace('\n','', regex=True)
tor.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
#casting both dataframes to type string
tor['PostalCode'] = tor['PostalCode'].astype(str)
latlong['PostalCode'] = latlong['PostalCode'].astype(str)
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge both dataframes

In [11]:
#merging both dataframes
tor = pd.merge(tor,latlong)
tor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Access FourSquare

In [12]:
CLIENT_ID = 'PNHNREUYZ5KTQZYVL1J0TBN3HCJFG4I4Q4WM5LMY0DWWPVDK' # your Foursquare ID
CLIENT_SECRET = 'YTTZ3SB4IMGCQYDDV1QVYV5MVPOEZPVVCNKIJC1CEI3KHUDU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PNHNREUYZ5KTQZYVL1J0TBN3HCJFG4I4Q4WM5LMY0DWWPVDK
CLIENT_SECRET:YTTZ3SB4IMGCQYDDV1QVYV5MVPOEZPVVCNKIJC1CEI3KHUDU


## Function that accesses foursquare api and searches for food places in Toronto, then formats them into a Dataframe

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            "4d4b7105d754a06374d81259")
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
tor_venues = getNearbyVenues(names=tor['Neighborhood'],
                                   latitudes=tor['Latitude'],
                                   longitudes=tor['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [15]:
tor_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
2,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
3,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
4,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place


## shape and unique categories in the Dataframe

In [16]:
tor_venues.shape


(1671, 7)

In [17]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 91 uniques categories.


## Preparing the data to be analyzed

In [18]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,...,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.drop(['Restaurant','Snack Place','Cafeteria','Café'],axis=1,inplace=True)
tor_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,...,Steakhouse,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Agincourt,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bayview Village,0.000000,0.000000,0.0,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.047619,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.095238,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.000000,0.000000
5,Berczy Park,0.000000,0.019231,0.0,0.0,0.000000,0.019231,0.019231,0.076923,0.019231,...,0.038462,0.057692,0.000000,0.000000,0.019231,0.019231,0.000000,0.038462,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.133333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000
8,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.500000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
tor_grouped.shape


(89, 88)

## get the top 5 venue for each neighborhood

In [21]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.33
1  Latin American Restaurant  0.33
2         Chinese Restaurant  0.33
3  Middle Eastern Restaurant  0.00
4                Pizza Place  0.00


----Alderwood, Long Branch----
                    venue  freq
0             Pizza Place  0.50
1            Burger Joint  0.25
2          Sandwich Place  0.25
3      Persian Restaurant  0.00
4  Peking Duck Restaurant  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0  Middle Eastern Restaurant  0.11
1        Fried Chicken Joint  0.11
2                      Diner  0.11
3              Deli / Bodega  0.11
4             Sandwich Place  0.11


----Bayview Village----
                        venue  freq
0            Asian Restaurant  0.25
1         Japanese Restaurant  0.25
2          Chinese Restaurant  0.25
3           Afghan Restaurant  0.00
4  Modern European Restaurant  0.00


----Bedford Park, Lawrence Manor Ea

##  Function that returns the most common venues in each neighborhood

In [22]:
# returns the most common venue in the neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
#for a better search many areas have 2 main categories
num_top_venues = 2

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
0,Agincourt,Breakfast Spot,Chinese Restaurant
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Sushi Restaurant
3,Bayview Village,Asian Restaurant,Chinese Restaurant
4,"Bedford Park, Lawrence Manor East",Pizza Place,Italian Restaurant


## K mean Clustering

In [48]:
# set number of clusters
kclusters = 7

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 3, 5, 5, 5, 5, 3, 5, 5, 5], dtype=int32)

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5.0,BBQ Joint,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Pizza Place,Portuguese Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,5.0,Bakery,Breakfast Spot
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,5.0,Vietnamese Restaurant,Hot Dog Joint
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,5.0,Sushi Restaurant,Japanese Restaurant


In [50]:
tor_ff=tor_merged.drop(['PostalCode','Borough'],axis=1)
tor_ff.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue
0,Parkwoods,43.753259,-79.329656,5.0,BBQ Joint,Fast Food Restaurant
1,Victoria Village,43.725882,-79.315572,3.0,Pizza Place,Portuguese Restaurant
2,"Regent Park, Harbourfront",43.654260,-79.360636,5.0,Bakery,Breakfast Spot
3,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,5.0,Vietnamese Restaurant,Hot Dog Joint
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,5.0,Sushi Restaurant,Japanese Restaurant


In [51]:
tor_ff.dropna(subset=['Cluster Labels'],axis=0,inplace=True)
tor_ff['Cluster Labels'].value_counts()

5.0    66
3.0    15
0.0     6
1.0     3
2.0     1
6.0     1
4.0     1
Name: Cluster Labels, dtype: int64

In [52]:
address = 'Toronto, CA'
# using geopy to find latitude and longitude of Toronto
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
#get the Latitude and longitude of toronto
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_ff['Latitude'], tor_ff['Longitude'], tor_ff['Neighborhood'], tor_ff['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [53]:
# Bakery
tor_ff.loc[tor_ff['Cluster Labels'] == 0, tor_ff.columns[[0] + list(range(4, tor_ff.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
21,Caledonia-Fairbanks,Bakery,Wings Joint
31,"Dufferin, Dovercourt Village",Pizza Place,Bakery
35,"East Toronto, Broadview North (Old East York)",Pizza Place,Bakery
38,"Kennedy Park, Ionview, East Birchmount Park",Filipino Restaurant,Chinese Restaurant
44,"Golden Mile, Clairlea, Oakridge",Bakery,Indian Restaurant
49,"North Park, Maple Leaf Park, Upwood Park",Bakery,Deli / Bodega


In [54]:
# pizza and Hakka Resturant
tor_ff.loc[tor_ff['Cluster Labels'] == 1, tor_ff.columns[[0] + list(range(4, tor_ff.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
5,"Islington Avenue, Humber Valley Village",Pizza Place,Hakka Restaurant
32,Scarborough Village,Pizza Place,Hakka Restaurant
72,"Willowdale, Willowdale West",Pizza Place,Bakery


In [55]:
tor_ff.loc[tor_ff['Cluster Labels'] == 2, tor_ff.columns[[0] + list(range(4, tor_ff.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
91,Rosedale,Japanese Restaurant,Wings Joint


In [56]:
# dominant Pizza Place with some breakfast, and asian resturants
tor_ff.loc[tor_ff['Cluster Labels'] == 3, tor_ff.columns[[0] + list(range(4, tor_ff.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
1,Victoria Village,Pizza Place,Portuguese Restaurant
8,"Parkview Hill, Woodbine Gardens",Pizza Place,Breakfast Spot
10,Glencairn,Pizza Place,Asian Restaurant
17,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pizza Place,Fried Chicken Joint
19,The Beaches,Pizza Place,Asian Restaurant
34,"Northwood Park, York University",Pizza Place,Caribbean Restaurant
50,Humber Summit,Pizza Place,Italian Restaurant
58,"Birch Cliff, Cliffside West",Pizza Place,Japanese Restaurant
63,"Runnymede, The Junction North",Pizza Place,Breakfast Spot
67,Davisville North,Pizza Place,Breakfast Spot


In [57]:
tor_ff.loc[tor_ff['Cluster Labels'] == 4, tor_ff.columns[[0] + list(range(4, tor_ff.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
61,Lawrence Park,Dim Sum Restaurant,Filipino Restaurant


In [58]:
tor_ff.loc[tor_ff['Cluster Labels'] == 5, tor_ff.columns[[0] + list(range(4, tor_ff.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
0,Parkwoods,BBQ Joint,Fast Food Restaurant
2,"Regent Park, Harbourfront",Bakery,Breakfast Spot
3,"Lawrence Manor, Lawrence Heights",Vietnamese Restaurant,Hot Dog Joint
4,"Queen's Park, Ontario Provincial Government",Sushi Restaurant,Japanese Restaurant
6,"Malvern, Rouge",Indian Restaurant,Fast Food Restaurant
7,Don Mills,Asian Restaurant,Italian Restaurant
9,"Garden District, Ryerson",Pizza Place,Sandwich Place
11,"West Deane Park, Princess Gardens, Martin Grov...",Filipino Restaurant,Wings Joint
13,Don Mills,Asian Restaurant,Italian Restaurant
14,Woodbine Heights,Hakka Restaurant,Diner


In [59]:
tor_ff.loc[tor_ff['Cluster Labels'] == 6, tor_ff.columns[[0] + list(range(4, tor_ff.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue
85,"Milliken, Agincourt North, Steeles East, L'Amo...",Asian Restaurant,Wings Joint


# Now I am interested in opening a Bakery that sells fresh pies, arabian sweets, and bread
### by looking at the data collected I can see that these neighborhoods have opened a resturant similar to what I want:(Dufferin, Dovercourt Village), (Caledonia-Fairbanks),  (East Toronto, Broadview North (Old East York)), and (Milliken, Agincourt North, Steeles East)
### Lets look into Dufferin, Dovercourt Village

In [66]:
Duff_data = tor_venues[tor_venues['Neighborhood']=='Dufferin, Dovercourt Village'].reset_index(drop=True)
Duff_data

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dufferin, Dovercourt Village",43.669005,-79.442259,Parallel,43.669516,-79.438728,Middle Eastern Restaurant
1,"Dufferin, Dovercourt Village",43.669005,-79.442259,Happy Bakery & Pastries,43.667050,-79.441791,Bakery
2,"Dufferin, Dovercourt Village",43.669005,-79.442259,Nova Era Bakery,43.669886,-79.437582,Bakery
3,"Dufferin, Dovercourt Village",43.669005,-79.442259,The Sovereign,43.673116,-79.440265,Café
4,"Dufferin, Dovercourt Village",43.669005,-79.442259,North Of Brooklyn Pizzeria,43.669467,-79.439335,Pizza Place
5,"Dufferin, Dovercourt Village",43.669005,-79.442259,Rush Hour,43.671183,-79.440313,Portuguese Restaurant
6,"Dufferin, Dovercourt Village",43.669005,-79.442259,241 Pizza,43.672948,-79.440971,Pizza Place
7,"Dufferin, Dovercourt Village",43.669005,-79.442259,Stop BBQ Chicken,43.665923,-79.438042,Brazilian Restaurant


In [68]:
#lets map these Boroughs
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

# create map of toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers of the toronto named boroughs to map 
for lat, lng, label in zip(Duff_data['Venue Latitude'], Duff_data['Venue Longitude'], Duff_data['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

The geograpical coordinate of North York are 43.6534817, -79.3839347.


## Lets search for more places in Dufferin Grove and see what comes up

In [75]:
LIMIT = 100 # Maximum is 100
city = ["Dufferin Grove, canada"]
results = {}

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4d4b7105d754a06374d81259")
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef0f8916001fe001b36a8b4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'geocode': {'what': '',
   'where': 'dufferin grove canada',
   'center': {'lat': 43.65766, 'lng': -79.42793},
   'displayString': 'Dufferin Grove, ON, Canada',
   'cc': 'CA',
   'longId': '72057594045798846'},
  'headerLocation': 'Dufferin Grove',
  'headerFullLocation': 'Dufferin Grove',
  'headerLocationGranularity': 'city',
  'query': 'food',
  'totalResults': 223,
  'suggestedBounds': {'ne': {'lat': 43.67017336280329,
    'lng': -79.4119304940833},
   'sw': {'lat': 43.64671655073645, 'lng': -79.44426277871952}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55610dcd498e1b69839a65d9'

In [77]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [78]:
venues = results['response']['groups'][0]['items']
    
duff_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.id' ,'venue.location.lat', 'venue.location.lng']
duff_venues =duff_venues.loc[:, filtered_columns]

# filter the category for each row
duff_venues['venue.categories'] = duff_venues.apply(get_category_type, axis=1)

# clean columns
duff_venues.columns = [col.split(".")[-1] for col in duff_venues.columns]

duff_venues.head()

,name,categories,id,lat,lng
0,Field Trip Cafe,Café,55610dcd498e1b69839a65d9,43.661260,-79.430450
1,Filosophy,Café,5a3aa18a5455b26eb31333d2,43.661879,-79.426471
2,La Bella Managua Restaurant,South American Restaurant,4b896cb7f964a520123532e3,43.662162,-79.424929
3,Bake Shoppe,Bakery,4b7b1b7ef964a520c6512fe3,43.654213,-79.423590
4,Terrazza,Italian Restaurant,4b638028f964a520c87e2ae3,43.659021,-79.422467


## Lets focus on Bakery places

In [84]:
duff_bakery=duff_venues[duff_venues['categories']=='Bakery']
duff_bakery

,name,categories,id,lat,lng
3,Bake Shoppe,Bakery,4b7b1b7ef964a520c6512fe3,43.654213,-79.423590
16,Pancho's Bakery,Bakery,4fd8c3c2e4b09507d5b9af86,43.660146,-79.435488
23,Home Baking Co.,Bakery,539df284498e79c67475ac1c,43.658988,-79.439864
29,Prairie Boy Bread,Bakery,564751d0498e7c1ca8add664,43.653410,-79.427447
76,Nova Era Bakery,Bakery,4af332d9f964a520aeeb21e3,43.649419,-79.422177
82,Through Being Cool Vegan Baking Co.,Bakery,5249fa1e11d2ae4d13bfc0d8,43.658455,-79.442291
85,Nova Era Bakery,Bakery,4b76ad66f964a520ec572ee3,43.661228,-79.429478
86,Courense Bakery,Bakery,4b765ddff964a520a7492ee3,43.660906,-79.430863
96,Nova Era Bakery,Bakery,4ad4c061f964a520a8f720e3,43.655001,-79.420351


## We can see that there are 9 bakeries with 3 of the 9 owned by Nova Era Bakery
## Lets plot and see where these bakeries are

In [83]:
#lets map these Boroughs
address = 'Dufferin Grove, canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

# create map of toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers of the toronto named boroughs to map 
for lat, lng, label in zip(duff_bakery['lat'], duff_bakery['lng'], duff_bakery['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

The geograpical coordinate of North York are 43.6536319, -79.4264389.


### From the plot we can see that there are bakeries on 3 streets lets investigate the bakeries to see where we can start our arabian sweet bakery.
### lets look at the rating of Nova Era Bakery (the one thats alone in the street)

In [85]:
# rating
venue_id = '4af332d9f964a520aeeb21e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.7


In [86]:
#number of tips
result['response']['venue']['tips']['count']

8

#### from these two reviews we can see they specialize in portuguese pastries
##### Please note I am Limited to only a few number of tips and ratings so I can only do 2 tips

In [87]:
limit = 2 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef0fce0be61c9001cbc6bf4'},
 'response': {'tips': {'count': 9,
   'items': [{'id': '54e228eb498e98141c3af866',
     'createdAt': 1424107755,
     'text': 'Best Portuguese pastries in Toronto!!',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/54e228eb498e98141c3af866',
     'photo': {'id': '54e228ed498e6d2101cebdcb',
      'createdAt': 1424107757,
      'source': {'name': 'Foursquare for iOS',
       'url': 'https://foursquare.com/download/#/iphone'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/119030646_Su4G2OyxXX-EiQAGdsDQQolGN-8jCdiTjUMDJOKF0lE.jpg',
      'width': 1920,
      'height': 1440,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/119030646_Su4G2OyxXX-EiQAGdsDQQolGN-8jCdiTjUMDJOKF0lE.jpg',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 1,
     'disagreeCount': 0,
     'todo': {'count': 

## Lets look into Bake Shoppe

In [88]:
# rating
venue_id = '4b7b1b7ef964a520c6512fe3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.4


In [89]:
#number of tips
result['response']['venue']['tips']['count']

12

In [90]:
limit = 2 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef100a61a4b0a001b9b12fb'},
 'response': {'tips': {'count': 12,
   'items': [{'id': '5262d77011d2e4e8b4ab362e',
     'createdAt': 1382209392,
     'text': 'Great selection of flavours and designs. Friendly staff and reasonable prices. Call to book a tasting appointment in advance or drop in to buy cupcakes or macaroons. Red velvet is the best!',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5262d77011d2e4e8b4ab362e',
     'photo': {'id': '5262d771498e4f06a42784a0',
      'createdAt': 1382209393,
      'source': {'name': 'Foursquare for Android',
       'url': 'https://foursquare.com/download/#/android'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/1012659_sWyL63AxenF6p4nUquI4qHec6aYJD_93VS1ytwMlKEE.jpg',
      'width': 960,
      'height': 720,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/1012659_sWyL63AxenF6p4nUquI4qHec6aYJD_93VS1ytwMlKEE.jpg',
 

## Lets look into Prairie Boy Bread

In [91]:
# rating
venue_id = '564751d0498e7c1ca8add664'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.5


In [92]:
result['response']['venue']['tips']['count']

3

In [93]:
limit = 3 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef101067828ae001bde6fda'},
 'response': {'tips': {'count': 3,
   'items': [{'id': '5667a78e498e6177df078d19',
     'createdAt': 1449633678,
     'text': 'Great bakery in Dufferin Grove. French baguettes and fresh loaves. Tea and other dry goods.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5667a78e498e6177df078d19',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 1,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '11859261',
      'firstName': 'Val',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/2VUOZYFRV110A5MJ.jpg'}},
     'authorInteractionType': 'liked'},
    {'id': '58f4d1c6dec1d65d998edb5e',
     'createdAt': 1492439494,
     'text': 'Delicious bakery with various breads plus tea and coffee.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/58f4d1c6dec1d65d998edb5e',
     'lang': 

## Pancho's Bakery

In [94]:
# rating
venue_id = '4fd8c3c2e4b09507d5b9af86'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.1


In [95]:
result['response']['venue']['tips']['count']

2

In [96]:
limit = 2 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef104431a4b0a001ba6efb8'},
 'response': {'tips': {'count': 2,
   'items': [{'id': '535e6ed5498e7ebdd98a6bfc',
     'createdAt': 1398697685,
     'text': 'Freshly made churros.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/535e6ed5498e7ebdd98a6bfc',
     'photo': {'id': '535e6eda11d28162ad79c8bb',
      'createdAt': 1398697690,
      'source': {'name': 'Foursquare Web', 'url': 'https://foursquare.com'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/75415759_q_KILKREKc7mQaqA1lm-xX5iK14eC-roPAyC42bT0dc.jpg',
      'width': 1500,
      'height': 1500,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/75415759_q_KILKREKc7mQaqA1lm-xX5iK14eC-roPAyC42bT0dc.jpg',
     'lang': 'en',
     'likes': {'count': 1,
      'groups': [{'type': 'others',
        'count': 1,
        'items': [{'id': '120332501',
          'firstName': 'Isserly',
          'photo': {'prefi

## Home Baking Co.

In [97]:
# rating
venue_id = '539df284498e79c67475ac1c'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.7


In [98]:
result['response']['venue']['tips']['count']

9

In [99]:
limit = 4 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef1014302a1720022936c26'},
 'response': {'tips': {'count': 9,
   'items': [{'id': '566f6a9b498e10f148e117ab',
     'createdAt': 1450142363,
     'text': 'Friendly faces, great coffee and incredible baking make for a terrific morning. Do go and say I sent you.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/566f6a9b498e10f148e117ab',
     'photo': {'id': '566f6a9b498e10f148e117aa',
      'createdAt': 1450142363,
      'source': {'name': 'Instagram', 'url': 'http://instagram.com'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/81374730_P_feZTX7321K1Zlat2S9ty7HU6xTx9eUfmkZZBV06OU.jpg',
      'width': 1080,
      'height': 1080,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/81374730_P_feZTX7321K1Zlat2S9ty7HU6xTx9eUfmkZZBV06OU.jpg',
     'lang': 'en',
     'likes': {'count': 1,
      'groups': [{'type': 'others',
        'count': 1,
        'items': [{'i

## Courense Bakery	

In [100]:
# rating
venue_id = '4b765ddff964a520a7492ee3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


In [101]:
result['response']['venue']['tips']['count']

5

In [102]:
limit = 4 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef104b0949393001bcec924'},
 'response': {'tips': {'count': 5,
   'items': [{'id': '53ef8c23498e012550f125e6',
     'createdAt': 1408207907,
     'text': 'This is where you get traditional Portuguese pastries. From breaded cod and shrimp to custard tarts.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/53ef8c23498e012550f125e6',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '63844',
      'firstName': 'Kevin',
      'lastName': 'O',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/63844_7Pbe5thJ_4BCEByzGUQIZ75WK_sQjaqDChnPxjbUUnFpD_0fG4asOQI5jtGftF6LIMV_thS80.jpg'}},
     'authorInteractionType': 'liked'},
    {'id': '4e69e0af18a83989ec358e64',
     'createdAt': 1315561647,
     'text': 'Open 24/7, 365.  Good espresso, bread and pastries.',
     'type': 'user',
  

## Through Being Cool Vegan Baking Co.	

In [103]:
# rating
venue_id = '5249fa1e11d2ae4d13bfc0d8'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.8


In [104]:
result['response']['venue']['tips']['count']

10

In [105]:
limit = 4 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef1025a02a172002296fe99'},
 'response': {'tips': {'count': 10,
   'items': [{'id': '524a47a911d279624621364b',
     'createdAt': 1380599721,
     'text': "Vegan donuts, brownies, cookies, scones, pretzels and prepared sandwiches and frozen pierogies, mac n' cheese, pot pies and enchiladas. Cash or debit.",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/524a47a911d279624621364b',
     'lang': 'en',
     'likes': {'count': 2,
      'groups': [{'type': 'others',
        'count': 2,
        'items': [{'id': '63135',
          'firstName': 'Jordan',
          'lastName': 'C',
          'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
           'suffix': '/DE0PXICNQBLAFTBA.jpg'}},
         {'id': '6725544',
          'firstName': 'Shawn',
          'lastName': 'P',
          'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
           'suffix': '/6725544_Wbx6X1UE_b4ZB0ruvshXHZLkwJDJEM631Z3yQAl9MB-C2CHVyhnm3JOlohpzy

## Summary

|  Bakery |Speciality   |  Rating | 
|---|---|---|
| Nova era           |  Portuguese  |  7.7 | 
|  Bake Shoppe       | Cakes, Cupcakes, and  Macaroons  | 8.4  |
|  Prairie Boy Bread |  French Breads |  8.1 | 
|Pancho's Bakery     |  Churros  |  8.7 | 
|  Home Baking Co    | Tarts and Ice cream  |  6.9 |  
|  Through Being Cool Vegan Baking Co. |  Vegan | 7.8  | 

### It should be a good place to open as there are no competetors in arabian sweets. However, we should make other type of sweets to attract customers as the area does not have arabian sweets. The area does have a Middle Eastern Resturant which should attract customers, it would be a good idea to open close by the Middle Eastern Resturant

In [109]:
duff_venues[duff_venues['categories']=='Middle Eastern Restaurant']

,name,categories,id,lat,lng
6,Tavoos Restaurant,Middle Eastern Restaurant,51572dc9e4b0b5f7ce9d0e13,43.652501,-79.433054
